In [1]:
import pandas as pd
import json
import matplotlib
import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
def read_csv(fname):
    f = lambda x : json.loads(x.replace("'","\""))
    df = pd.read_csv(fname, converters={'sensor_type': f}, index_col=0)
    return df

In [3]:
def save_graph(plot, fname='saved.png'):
    fig = plot.get_figure()
    fig.savefig(fname)

In [4]:
df = read_csv('test_record-20170425.csv')
df = df[df['type'] == 'kNN']
df.sort_values(by=['auc'], ascending=False).head()

fig = plt.figure()

plot = df[
#    (df['ma_window'] == 1) &
    (df['model_idx'] == 1) &
#    (df['ms_interval'] == 20) & 
    (df['n_record'] == 1) & 
    (df['sensor_type'].apply(lambda x: len(x) == 2))
].plot.scatter(x='n_train', y='auc')

save_graph(plot, fname='saved.png')